In [ ]:
import os
import datetime
try:
    __file__ = os.path.split(__file__)[-1]
    __file__ = os.path.splitext(__file__)[0]
except NameError:
    __file__ = '000-simple-template'
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S_%f')
print(timestamp)

import sqlite3
import time
import pprint

from fiutils.db import *
from fiutils.params import *

from contextlib import closing
from types import SimpleNamespace

from tqdm.auto import trange, tqdm

db_name = f'data/{__file__}.db'
table_name = f'tab_{timestamp}'

with closing(sqlite3.connect(db_name)) as db:
    hist = db_get_hist(db, table_name)
    display(hist)

params = [
    Parameter('scan', max=500_000, itype='range'),
    Parameter('glitch_delay_ns', 10, 10_000),
    Parameter('glitch_time_ns', 50, 1200),
    Parameter('glitch_v', 0.0, 1.5, dtype='float'),
]

# You can reconstruct a Parameter object with Parameter(**loaded_json)
pdump(params, 'params/' + os.path.splitext(__file__)[0] + '-' + timestamp + '.json')

with closing(sqlite3.connect(db_name)) as db:
    db_setup(db)

    progress = tqdm(enumerate(pproduct(params)), total=ptotal(params), mininterval=1)
    for idx, settings in progress:
        try:
            t0 = time.time_ns()
            params = SimpleNamespace(**settings)
            result = settings.copy()
            result['timestamp'] = time.time()
            result['verdict'] = 'FAIL'

            # Do smart things there
            if (params.glitch_v * params.glitch_time_ns) > 500:
                result['verdict'] = 'MUTE'
            if 400 < (params.glitch_v * params.glitch_time_ns) < 500:
                if 3000 < params.glitch_delay_ns < 5000:
                    result['verdict'] = 'GLITCH'
                else:
                    result['verdict'] = 'MUTE'
            
            result['iter_t'] = time.time_ns() - t0

            try:
                hist[result['verdict']] += 1
            except KeyError:
                hist[result['verdict']] = 1

            # If you can do >1000 it/s, removing prints will help
            progress.write(pprint.pformat(result))
            progress.write(pprint.pformat(hist))

            db_append_row(db, table_name, result)

        except KeyboardInterrupt:
            try:
                x = input('[c]ontinue or [q]uit?>\n')
            except:
                break
            if x == 'c':
                continue
            else:
                break

print('BYE')